In [ ]:
# 安装依赖 (Colab 通常已有 torch/PIL)
!pip install spandrel -q
!pip install tqdm -q

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
import gc

# ============== 配置区 ==============
# 模型路径 (自动下载 4x-UltraSharp)
MODEL_URL = "https://huggingface.co/datasets/Kizi-Art/Upscale/resolve/main/4x-UltraSharp.pth"
MODEL_PATH = "/content/models/4x-UltraSharp.pth"

# 输入输出目录
INPUT_DIR = "/content/input"   # 把图片放这里
OUTPUT_DIR = "/content/output" # 输出到这里

# 放大设置
TARGET_SCALE = 8             # 目标放大倍数 (8 或 16)
MAX_SIDE_LENGTH = 8192       # 最大边长限制
TILE_SIZE = 512              # 分块大小 (显存不足可降低)
TILE_OVERLAP = 32            # 分块重叠

# 设备
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
USE_HALF = True              # 使用半精度 (省显存)

print(f"使用设备: {DEVICE}")

In [ ]:
# 下载模型
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

if not os.path.exists(MODEL_PATH):
    print("正在下载 4x-UltraSharp 模型...")
    !wget -q -O "{MODEL_PATH}" "{MODEL_URL}"
    print("下载完成!")
else:
    print("模型已存在")

In [ ]:
# 核心函数: 图像转换
def pil_to_torch_bgr(img: Image.Image) -> torch.Tensor:
    """PIL RGB -> Torch BGR [0,1]"""
    arr = np.array(img.convert("RGB"))
    arr = arr[:, :, ::-1]  # RGB -> BGR
    arr = np.transpose(arr, (2, 0, 1))  # HWC -> CHW
    arr = np.ascontiguousarray(arr) / 255.0
    return torch.from_numpy(arr).float()

def torch_bgr_to_pil(tensor: torch.Tensor) -> Image.Image:
    """Torch BGR [0,1] -> PIL RGB"""
    if tensor.ndim == 4:
        tensor = tensor.squeeze(0)
    arr = tensor.float().cpu().clamp_(0, 1).numpy()
    arr = 255.0 * np.moveaxis(arr, 0, 2)  # CHW -> HWC
    arr = arr.round().astype(np.uint8)
    arr = arr[:, :, ::-1]  # BGR -> RGB
    return Image.fromarray(arr, "RGB")

In [ ]:
# 核心函数: 分块放大 (避免显存爆炸)
def tiled_upscale(
    img: Image.Image,
    model,
    tile_size: int = 512,
    tile_overlap: int = 32,
    scale: int = 4,
    device: str = "cuda",
    dtype = torch.float16
) -> Image.Image:
    """分块放大图像"""
    w, h = img.size
    
    # 如果图像小于 tile_size，直接整张处理
    if w <= tile_size and h <= tile_size:
        tensor = pil_to_torch_bgr(img).unsqueeze(0).to(device=device, dtype=dtype)
        with torch.inference_mode():
            output = model(tensor)
        return torch_bgr_to_pil(output)
    
    # 分块处理
    stride = tile_size - tile_overlap
    h_tiles = list(range(0, h - tile_size, stride)) + [max(0, h - tile_size)]
    w_tiles = list(range(0, w - tile_size, stride)) + [max(0, w - tile_size)]
    
    # 输出画布
    out_h, out_w = h * scale, w * scale
    result = torch.zeros((3, out_h, out_w), dtype=torch.float32)
    weight = torch.zeros((1, out_h, out_w), dtype=torch.float32)
    
    total_tiles = len(h_tiles) * len(w_tiles)
    
    with tqdm(total=total_tiles, desc="分块放大", leave=False) as pbar:
        for y in h_tiles:
            for x in w_tiles:
                # 裁剪 tile
                tile = img.crop((x, y, x + tile_size, y + tile_size))
                tensor = pil_to_torch_bgr(tile).unsqueeze(0).to(device=device, dtype=dtype)
                
                with torch.inference_mode():
                    output = model(tensor).squeeze(0).float().cpu()
                
                # 计算输出位置
                out_y, out_x = y * scale, x * scale
                out_tile_h, out_tile_w = output.shape[1], output.shape[2]
                
                # 累加结果 (重叠区域会被平均)
                result[:, out_y:out_y+out_tile_h, out_x:out_x+out_tile_w] += output
                weight[:, out_y:out_y+out_tile_h, out_x:out_x+out_tile_w] += 1
                
                pbar.update(1)
    
    # 平均重叠区域
    result = result / weight.clamp(min=1)
    
    return torch_bgr_to_pil(result)

In [ ]:
# 加载模型
import spandrel

print("正在加载模型...")
model_descriptor = spandrel.ModelLoader(device=DEVICE).load_from_file(MODEL_PATH)
model = model_descriptor.model

if USE_HALF and model_descriptor.supports_half:
    model.half()
    print("使用 FP16 半精度")

model.eval()
MODEL_SCALE = model_descriptor.scale  # 通常是 4
print(f"模型加载完成! 单次放大倍数: {MODEL_SCALE}x")

In [ ]:
# 多次迭代放大
def multi_scale_upscale(
    img: Image.Image,
    model,
    target_scale: float,
    model_scale: int = 4,
    max_side: int = 8192,
    tile_size: int = 512,
    tile_overlap: int = 32,
    device: str = "cuda"
) -> Image.Image:
    """
    多次迭代放大到目标倍数
    例如: 8x = 4x + 4x (两次), 16x = 4x + 4x + 4x + 4x (四次)
    """
    dtype = torch.float16 if USE_HALF else torch.float32
    current_img = img
    current_scale = 1.0
    
    iteration = 0
    while current_scale < target_scale:
        iteration += 1
        w, h = current_img.size
        
        # 检查是否会超过最大边长
        next_w, next_h = w * model_scale, h * model_scale
        if max(next_w, next_h) > max_side:
            print(f"  警告: 下一次放大会超过 {max_side}px，停止迭代")
            break
        
        print(f"  迭代 {iteration}: {w}x{h} -> {next_w}x{next_h}")
        
        current_img = tiled_upscale(
            current_img,
            model,
            tile_size=tile_size,
            tile_overlap=tile_overlap,
            scale=model_scale,
            device=device,
            dtype=dtype
        )
        
        current_scale *= model_scale
        
        # 清理显存
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    return current_img

In [ ]:
# 批量处理
def batch_upscale(
    input_dir: str,
    output_dir: str,
    model,
    target_scale: float = 8,
    max_side: int = 8192,
    tile_size: int = 512,
    tile_overlap: int = 32
):
    """批量处理目录中的所有图片"""
    supported_formats = {".png", ".jpg", ".jpeg", ".webp", ".bmp"}
    
    # 获取所有图片文件
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    image_files = [
        f for f in input_path.iterdir()
        if f.suffix.lower() in supported_formats
    ]
    
    if not image_files:
        print(f"在 {input_dir} 中没有找到图片文件")
        return
    
    print(f"找到 {len(image_files)} 张图片")
    print(f"目标放大倍数: {target_scale}x, 最大边长: {max_side}px")
    print("="*50)
    
    for i, img_path in enumerate(image_files, 1):
        print(f"\n[{i}/{len(image_files)}] 处理: {img_path.name}")
        
        try:
            # 加载图片
            img = Image.open(img_path).convert("RGB")
            orig_w, orig_h = img.size
            print(f"  原始尺寸: {orig_w}x{orig_h}")
            
            # 放大
            result = multi_scale_upscale(
                img,
                model,
                target_scale=target_scale,
                model_scale=MODEL_SCALE,
                max_side=max_side,
                tile_size=tile_size,
                tile_overlap=tile_overlap,
                device=DEVICE
            )
            
            # 保存 (保留原文件名)
            output_file = output_path / f"{img_path.stem}_upscaled.png"
            result.save(output_file, "PNG")
            
            final_w, final_h = result.size
            print(f"  最终尺寸: {final_w}x{final_h}")
            print(f"  已保存: {output_file.name}")
            
            # 清理
            del img, result
            gc.collect()
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
            
        except Exception as e:
            print(f"  错误: {e}")
            continue
    
    print("\n" + "="*50)
    print("处理完成!")

In [ ]:
# 上传图片到 input 目录 (Colab)
from google.colab import files

print("请上传要放大的图片...")
uploaded = files.upload()

for filename, data in uploaded.items():
    with open(os.path.join(INPUT_DIR, filename), 'wb') as f:
        f.write(data)
    print(f"已保存: {filename}")

In [ ]:
# 开始批量放大!
batch_upscale(
    input_dir=INPUT_DIR,
    output_dir=OUTPUT_DIR,
    model=model,
    target_scale=TARGET_SCALE,
    max_side=MAX_SIDE_LENGTH,
    tile_size=TILE_SIZE,
    tile_overlap=TILE_OVERLAP
)

In [ ]:
# 下载结果 (可选)
import shutil

# 打包输出目录
shutil.make_archive("/content/upscaled_images", 'zip', OUTPUT_DIR)
files.download("/content/upscaled_images.zip")

## 📝 说明

**依赖极简**:
- `spandrel`: 用于加载 ESRGAN 类模型 (替代完整 SD WebUI)
- `torch`, `PIL`, `numpy`: 基础依赖

**优势**:
1. 无需加载 Stable Diffusion 模型 (省显存/时间)
2. 直接使用 spandrel 加载 .pth 模型
3. 支持分块处理 (tile) 避免显存爆炸
4. 多次迭代放大支持 8x/16x
5. 保留原文件名

**显存建议**:
- T4 (16GB): TILE_SIZE=512 可用
- V100/A100: TILE_SIZE=768 更快
- 显存不足: 降低 TILE_SIZE 到 256